# Introduction to Langkit

In [ ]:
!pip install langkit[all]

In [ ]:
from langkit import llm_metrics
import whylogs as why

why.init()
schema = llm_metrics.init()

In [ ]:
from langkit.whylogs.samples import load_chats, show_first_chat

chats = load_chats()
len(chats)

In [ ]:
show_first_chat(chats)

In [ ]:
results = why.log(chats,name='langkit-demo',schema=schema)

# Working with WhyLabs & Telemetry Agents

In [ ]:
!pip install transformers
!pip install langkit[all]

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
def gpt_model(prompt):

  # Encode
  input_ids = tokenizer.encode(prompt, return_tensors='pt')

  # Generate a response
  output = model.generate(input_ids, max_length=100, temperature=0.8,
                          do_sample=True, pad_token_id=tokenizer.eos_token_id)

  # Decode

  response = tokenizer.decode(output[0], skip_special_tokens=True)
  pandr = {
      "prompt": prompt,
      "response": response
  }

  return pandr

In [ ]:
pandr = gpt_model("Tell me a story about a cute cat")
print(pandr)

In [ ]:
!pip install langkit[all]

In [ ]:
from langkit import llm_metrics # alternatively use 'light_metrics'
import whylogs as why
import pandas as pd

# Set to show all columns in dataframe
pd.set_option("display.max_columns", None)

# Note: llm_metrics.init() downloads models so this is slow first time.
schema = llm_metrics.init()

In [ ]:
profile = why.log(pandr, schema=schema).profile()

In [ ]:
profview = profile.view()
profview.to_pandas()

# Demo with Multiple Prompts

In [ ]:
prompts = ["What is ML?",
           "Tell me a joke on cats.",
           "Who won the world series in 2012?"]

In [ ]:
for num, prompt in enumerate(prompts):

  pandr = gpt_model(prompt)

  # initial profile schema on first profile
  # if num == 0:
  #   profile = why.log(pandr, schema=schema).profile()
  profile.track(pandr)

In [ ]:
profview = profile.view()
profview.to_pandas()

# Push Metrics to WhyLabs

In [ ]:
import os
# set authentication & project keys
os.environ["WHYLABS_DEFAULT_ORG_ID"] = 'org-rdPzFz'
os.environ["WHYLABS_API_KEY"] = '6MT1PHbNIH.TMEKdtTmiZQxhhZuO6CAI47xpYruaHZ2Rht2VqvygeO370gwCcSnD:org-rdPzFz'
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = 'model-9'

In [ ]:
from whylogs.api.writer.whylabs import WhyLabsWriter
from langkit import llm_metrics
import whylogs as why


schema = llm_metrics.init()

In [ ]:
telemetry_agent = WhyLabsWriter()
profile = why.log(pandr, schema=schema)
telemetry_agent.write(profile.view())

In [ ]:
#Filling the data with WhyLabs

prompt_lists = [
    ["How can I create a new account?", "Great job to the team", "Fantastic product, had a good experience"],
    ["This product made me angry, can I return it? Give a phone number to call 800-987-6543", "You dumb and smell bad", "I hated the experience, and I was over charged"],
    ["This seems amazing, could you share the pricing?", "Incredible site, could we setup a call?", "Hello! Can you kindly guide me through the documentation?"],
    ["This looks impressive, could you provide some information on the cost?", "Stunning platform, can we arrange a chat?", "Hello there! Could you assist me with the documentation?"],
    ["This looks remarkable, could you tell me the price range?", "Fantastic webpage, is it possible to organize a call?", "Greetings! Can you help me with the relevant documents?"],
    ["This is great, Ilove it, could you inform me about the charges?", "love the interface, can we have a teleconference?", "Hello! Can I take a look at the user manuals?"],
    ["This seems fantastic, how much does it cost?", "Excellent website, can we setup a call?", "Hello! Could you help me find the resource documents?"]
]

In [ ]:
import datetime


telemetry_agent = WhyLabsWriter()
all_prompts_and_responses = []  # This list will store all the prompts and responses.


for i, day in enumerate(prompt_lists):
  telemetry_agent = WhyLabsWriter()
  # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
  dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)
  for prompt in day:
    pandr = gpt_model(prompt)
    profile = why.log(pandr, schema=schema)

     # Save the prompt and its response in the list.
    all_prompts_and_responses.append(pandr)

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    telemetry_agent.write(profile.view())

In [ ]:
all_prompts_and_responses

In [ ]:
!pip install gradio

In [ ]:
import datetime
import gradio as gr

In [ ]:
def llm_response(message, days_to_subtract):

      dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=days_to_subtract)

      telemetry_agent = WhyLabsWriter()
      prompt_and_response = gpt_model(message)
      # prompt_and_response = gpt_model(message)
      profile = why.log(prompt_and_response, schema=schema).profile()

      profile.set_dataset_timestamp(dt)
      telemetry_agent.write(profile.view())

      # Could add gaurdrail here! (Next section)

      response = prompt_and_response["response"]

      return response


iface = gr.Interface(
    fn=llm_response,
    inputs=["text", gr.Slider(minimum=0, maximum=6, step=1,  value="Number of days to subtract from todays date (0=today, 6=week ago)")],
    outputs="text",
    live=False,
)

iface.launch()

# Guardrails and Validation

In [ ]:
import os
import whylogs as why
import langkit.toxicity
import pandas as pd

from langkit import llm_metrics

print("downloading models and initialized metrics...")
text_metrics_schema = llm_metrics.init()

# Set to show all columns in dataframe
pd.set_option("display.max_columns", None)

In [ ]:
print("downloading models and initialized metrics...")
text_metrics_schema = llm_metrics.init()


In [ ]:
def is_not_toxic(prompt_message):

    profile = why.log({"prompt": prompt_message}, schema=text_metrics_schema).profile().view()

    profile_dict = profile.get_column('prompt.toxicity').to_summary_dict()
    tox_max = profile_dict['distribution/max']

    print(f'Toxic score: {tox_max}')

    if tox_max > 0.5:
      return False
    else:
      return True

In [ ]:
is_not_toxic("Do you like fruit?")

In [ ]:
is_not_toxic("You dumb and smell bad")

In [ ]:
user_prompt = "Do you like fruit?"

if is_not_toxic(user_prompt):
  pandr = gpt_model(user_prompt)
  print(pandr['response'])

else:
  print("As a large language model...")

In [ ]:
user_prompt = "You dumb and smell bad"

if is_not_toxic(user_prompt):
  pandr = gpt_model(user_prompt)
  print(pandr['response'])

else:
  print("As a large language model...")